In [1]:
import cv2
import numpy as np
import time
import os
import glob

def simulate_daltonism(image, daltonism_type):
    image_copy = image.copy()
    if daltonism_type == "Deutéranopie":
        image_copy[:, :, 1] = 0
    return image_copy

def analyser_pixels(image):
    rouge_count = 0
    vert_count = 0
    bleu_count = 0
    daltonien_count = 0
    total_pixels = image.shape[0] * image.shape[1]

    for i in range(image.shape[0]):
        for j in range(image.shape[1]):
            pixel = image[i, j]
            bleu, vert, rouge = pixel
            if rouge > vert and rouge > bleu:
                rouge_count += 1
            elif vert > rouge and vert > bleu:
                vert_count += 1
                daltonien_count += 1
            elif bleu > rouge and bleu > vert:
                bleu_count += 1

    rouge_percentage = (rouge_count / total_pixels) * 100
    vert_percentage = (vert_count / total_pixels) * 100
    bleu_percentage = (bleu_count / total_pixels) * 100
    daltonien_percentage = (daltonien_count / total_pixels) * 100

    return rouge_percentage, vert_percentage, bleu_percentage, daltonien_percentage

def traiter_image(image, chemin_image):
    start_time = time.time()

    rouge, vert, bleu, daltonien = analyser_pixels(image)
    nom_image = os.path.basename(chemin_image)

    print(f"\n {nom_image} | Daltonien: {daltonien:.2f}%")

    if daltonien > 5:
        print("Image non visible pour les daltoniens")
        dossier_sortie = "output/images_non_visibles_par_daltoniens_transformé"
        os.makedirs(dossier_sortie, exist_ok=True)
        modifiee = simulate_daltonism(image, "Deutéranopie")
        cv2.imwrite(os.path.join(dossier_sortie, f"deuteranopie_{nom_image}"), modifiee)
    else:
        print("Image visible pour les daltoniens")
        dossier_sortie = "output/images_visibles_par_daltoniens_de_base"
        os.makedirs(dossier_sortie, exist_ok=True)
        cv2.imwrite(os.path.join(dossier_sortie, nom_image), image)

    print(f"Temps CPU : {time.time() - start_time:.4f} sec")

def main():
    dossier_images = "Input"
    chemins_images = glob.glob(os.path.join(dossier_images, "*.jpg"))

    if not chemins_images:
        print("Aucun fichier .jpg trouvé dans Input/")
        return

    total_start = time.time()
    
    for chemin in chemins_images:
        image = cv2.imread(chemin)
        if image is None:
            print(f"Erreur de lecture : {chemin}")
            continue
        traiter_image(image, chemin)


    total_time = time.time() - total_start
    print(f"\nTemps total pour toutes les images : {total_time:.2f} secondes")
    
if __name__ == "__main__":
    main()


 2_deut.jpg | Daltonien: 10.96%
Image non visible pour les daltoniens
Temps CPU : 0.3888 sec

 4_deut.jpg | Daltonien: 26.47%
Image non visible pour les daltoniens
Temps CPU : 0.3469 sec

 8_visible.jpg | Daltonien: 4.29%
Image visible pour les daltoniens
Temps CPU : 0.1447 sec

 3_deut.jpg | Daltonien: 30.84%
Image non visible pour les daltoniens
Temps CPU : 0.3579 sec

 1_deut.jpg | Daltonien: 48.85%
Image non visible pour les daltoniens
Temps CPU : 0.1945 sec

Temps total pour toutes les images : 1.46 secondes
